In [1]:
import pandas as pd
from collections import Counter
import numpy as np
from opencc import OpenCC
import jieba
import random

from keras.models import Sequential
from keras.layers import Dense
from keras.layers.merge import add
from keras.layers import LSTM,Bidirectional, Dropout,GRU
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras import regularizers
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


In [2]:
def performance(y, y_hat):
    print("accuracy: {}".format(accuracy_score(y, y_hat,normalize=False)))
    print("precision: {}".format(precision_score(y, y_hat,average='micro')))
    print("recall: {}".format(recall_score(y, y_hat)))
    print("roc_auc: {}".format(roc_auc_score(y, y_hat)))

# 读取数据

In [3]:
data = pd.read_csv('D:/senior/aiCourse/dataSource/comment_classification/train/sentiment_analysis_trainingset.csv',encoding='UTF-8')

In [4]:
data.count

<bound method DataFrame.count of             id                                            content  \
0            0  "吼吼吼，萌死人的棒棒糖，中了大众点评的霸王餐，太可爱了。一直就好奇这个棒棒糖是怎么个东西，...   
1            1  "第三次参加大众点评网霸王餐的活动。这家店给人整体感觉一般。首先环境只能算中等，其次霸王餐提...   
2            2  "4人同行 点了10个小吃\n榴莲酥 榴莲味道不足 松软 奶味浓\n虾饺 好吃 两颗大虾仁\...   
3            3  "之前评价了莫名其妙被删 果断继续差评！ 换了菜单 价格更低 开始砸牌子 但套餐还是有150...   
4            4  "出乎意料地惊艳，椰子鸡清热降火，美容养颜，大大满足了爱吃火锅怕上火星人。椰子冻是帅帅的老板...   
5            5  "烤鸭使用的蘸酱很地道，觉得烤鸭好不好吃在很大程度上都取决于这个酱料是否正宗。个人感觉他们家...   
6            6  "在好友圈很红的原因很诡异.竟然基本都是我外地的朋友转发的.不远千里跑来厦门买回去吃.吃完各...   
7            7  "中午饭点人比较多需要排队～2点左右去吃的刚好可以用团购～也不用排队～就是平安夜店员说的都不...   
8            8  "跟朋友相约25号看港囧，首映当天排片率超高，为了提前订好票，选了文投国际影城，为了看电影我...   
9            9  "首先感谢大众点评再次抽中我免费品尝火锅，同时也感谢古蜀雒味三星堆主题火锅商家提供这个平台，...   
10          10  "这次必须给全五分！因为太完美了！整个就餐体验相当棒！\n口味：菜品做的都很干净！有图有真相...   
11          11  "突然特别想吃点重口味的，就和老公商量了去吃辣府。以前都不太敢吃辣的。没想到不是周末都要排队...   
12          12  "和闺蜜相约欧亚达鱼酷，三个人竟然喜欢不同口味，最后只得放弃我最爱的黄金酸辣，吃的油泼香辣口...   
1

In [5]:
#填充空白格
data['content'] = data['content'].fillna('')

In [6]:
sample = 10000
random_indices = np.random.choice(np.arange(len(data['content'])), sample)
sub_data = data.iloc[random_indices]

In [7]:
sub_data.head()

,id,content,location_traffic_convenience,location_distance_from_business_district,location_easy_to_find,service_wait_time,service_waiters_attitude,service_parking_convenience,service_serving_speed,price_level,...,environment_decoration,environment_noise,environment_space,environment_cleaness,dish_portion,dish_taste,dish_look,dish_recommendation,others_overall_experience,others_willing_to_consume_again
13069,13069,"""去了3次了 总体感觉还可以 我买的卷是很优惠的那种 现在的价格 对我来说 有点不太划算 东...",-2,-2,-2,-2,0,-2,-2,1,...,-2,-2,1,-1,1,0,-2,-2,1,-2
99289,99289,"""古色古香的装修，外面还挂了一面旗子，上面写着大大的繁体“面”字。\n除了卖苏面，生煎和小笼...",-2,-2,-2,-2,-2,-2,-2,0,...,1,-2,-2,-2,1,0,-2,-2,1,-2
60885,60885,"""两大一小点了一桌才100出头确实不贵，口味就一般了。主食炒饭第一个上了，冷菜还没也是服了。...",-2,-2,-2,-1,-2,-2,-2,0,...,1,-2,-2,-2,0,0,-2,-2,1,-2
35094,35094,"""在苏州竟然有家那么好吃的苏帮菜，已经第二次来吃了，味道依旧那么好。最令我难以忘怀的还是这家...",-2,-2,-2,-2,-2,-2,-2,-2,...,-2,-2,-2,-2,-2,1,1,-2,1,-2
90026,90026,"""已经是第二次去了，依然觉得是吃过最好吃的印度菜。就像不到中国不知道涮肉能分出各种派别一样，...",-2,-2,-2,-2,0,-2,-2,-1,...,-2,-2,-2,-2,-2,1,-2,-2,1,1


# 预处理，主要处理content字段

In [8]:

def is_CN_char(ch):
    return ch >= u'\u4e00' and ch <= u'\u9fa5'

def cut(string):
    return list(jieba.cut(string))

def get_stopwords(filename = "D:/senior/aiCourse/dataSource/stop_word.txt"):
    stopwords_dic = open(filename, encoding= 'utf-8')
    stopwords = stopwords_dic.readlines()
    stopwords = [w.strip() for w in stopwords]
    stopwords_dic.close()
    return stopwords

def convert2simple(word):
    openCC = OpenCC('tw2sp')
    return openCC.convert(word)

In [9]:
stopwords = get_stopwords()

In [10]:
def clean_sentence(sentence):
    stopwords = get_stopwords()
    sentence = ''.join(filter(is_CN_char,sentence))
    #print(sentence)
    
    
    sentence = convert2simple(sentence)
    words = [w for w in cut(sentence) if len(w)>0 and w not in stopwords]
    #print(words)
    words = ' '.join(words)
    return words  

In [11]:
clean_sentence("哈哈哈哈哈")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Andywong\AppData\Local\Temp\jieba.cache
Loading model cost 0.584 seconds.
Prefix dict has been built succesfully.


'哈哈哈 哈哈'

In [16]:
sub_data['content'][90026]

'"已经是第二次去了，依然觉得是吃过最好吃的印度菜。就像不到中国不知道涮肉能分出各种派别一样，不吃印度菜只会觉得咖喱就是棕色重口味的菜汤。。。\n他家胜在咖喱种类非常丰富，从大的种类分为羊肉、海鲜和素菜（印度人把牛当神，所以不吃牛肉），而即便是选择两种素咖喱也会从汤汁中吃出细微的区别。\n咖喱的价格基本稳定在50米一份左右，“不便宜”但味道的确有说服力。现做的蒜香和黄油烤饼也都18左右，没办法，谁让人家是现做的味道好呢。。。如果不太会吃印度菜，老板会为你简单介绍讲解\n中午提供自助，有机会我会去尝试的。"'

In [12]:
result = sub_data['content'].apply(clean_sentence)

KeyboardInterrupt: 

In [13]:
result.tolist()

['该店 位于 莲池 南大街 号 裕华 食品城 对 公交车 路路 路路 交通 方便 新开 不算 太火 不远 钟楼 商厦 地下 稻香村 糕点 保定 稻香村 糕点 总店 在 保定 商场 后身 西大街 古色古香 建筑 二层 小楼 屋内 免费 无线 一楼 柜台 二楼 可以 休闲 最 多 西大街 总店 历史悠久 保定 老人 对 记忆 有大 面包 椰蓉 红豆 鸡蛋 味道 钟楼 大 面包 拼 各有特色 可以 预定 生日蛋糕 各种 样式 可 供选择 各种 样式 糕点 如 饼干 月饼 快到 八月 十五 大家 挑 月饼 木糖醇 月饼 白砂糖 月饼 酥皮 硬皮 改良 口味 凤梨 五仁 莲蓉 枣泥 豆沙 巧克力 肉松 薄 皮大馅 高中低档 各种 包装 可以 作为 馈赠 亲朋 好 礼',
 '我团 双人 套餐 休息日 没什幺 事儿 妈妈 没 时间 这回 终于 可以 套餐 里面 有水 煎肉 组合 黑牛 牛 五花 霜降 雪花 肉 黑牛 胸口 菜量 不多肉 口感 不错 肉 薄 但 喜欢 蔬菌 组合 韭菜 油麦 菜 金针菇 香菇 不错 素食 对 来说 重要 喜欢 芝士 玉米粒 芝士 喜欢 可是 妈妈 不 喜欢 年龄 大 人 不 喜欢 田园 蔬菜 沙拉 妈妈 喜欢 不 喜欢 沙拉 少 喜欢 多多 沙拉酱 那才 好吃 不过 很胖 哈哈 章鱼 上菜 时 已经 在 锅 以为 没上 呢 味道 嘛 好极了 营养 蟹 籽 饭 当时 不 特意 拌 服务员 告诉 一会儿 放到 锅 里面 才 傻傻的 不去 动 它 真笨 阿布 基诺 布丁 不 好吃 至少 不 喜欢 在家 做 甜品 换 别的 好 总体 这次 中饭 不错 妈妈 没吃过 这样 烤肉 呢 不错 方式 味道 烤肉 入味 后期 时 腻 免费 柠檬水 真的 不 环境 装修 非常 喜欢 本人 非常 喜欢 猪猪 东西 根猪 喜欢 环境 非常 喜欢 卡通 爱 非常 特色 临走时 妈妈 下次 还要 呢 不过 不 套餐 单 点点 些 没吃过 全都 尝尝 嘿嘿',
 '不错 幸运 霸王餐 开心 带 开始 时间 快 过期 才 大概 晚上 左右 人流 很多 但是 一看 爸爸 得 多分钟 才能 在 沙发 坐 会儿 店员 真的 亲切 还给 送来 水 喝边 坐 欣赏 装修 真的 精致 好 桌子 木头 挺大 人 开始 多 爆满 老爸 人家 霸王餐 店家 服务态度 特别 好 一点 霸王餐 爱答

In [14]:
sub_data['content'][:10]

22444     "该店位于莲池南大街536号，裕华食品城对过，这里有公交车61路，28路，35路，7路，交通...
90903     "我团的是双人套餐，休息日没什么事儿和妈妈一起去吃，一直想去就是没时间，这回终于可以吃上了。...
101206    "感觉很不错哦    很幸运 中了霸王餐  感觉很开心   带着由于开始的时候没有时间去  ...
32077     "Tgi Friday means Thank God, It's Friday!\n貌似在...
41349     "因为刚好肚子饿就问了租车的老板哪里好吃，他指给我们的，就在租车对面，我们刚去的时候是这家店...
101478    "位于衡山路酒吧一条街上很多小资的店，这家就是其中之一，下午茶，正餐于一身。。感觉下午茶生意...
6970      "看完碟中碟，就想着糊吃海喝一顿了。海岸城吃的地方太多了，我们就挑了这家了，哈哈！这家主色调...
2344      "我又来五道口这家了，路过进去看看都成为习惯了^_^每周路过至少两次的我是不是去的很频繁？\...
52750     "今天第一次去吃好好先生。。到的时候已经9点了。，买了398的团购还点了一些东西。\n398...
73804     "出差来合肥，不了解当地美食，偌大的银泰城中午人不多，看了点评来的。因为特别想吃烤鸭，果然没...
Name: content, dtype: object

In [15]:
from wordcloud import WordCloud

In [16]:
wc = WordCloud(
        background_color='white', font_path='D://senior/aiCourse/dataSource/SourceHanSerifSC-Regular.otf'       
    )

In [17]:
worddata = ''.join(sub_data['content'].tolist())

In [18]:
wc.generate_from_text(worddata)

In [19]:
wc.to_file('comment_wc.png')

In [20]:
def word_to_id(vocab):
    counts = Counter(vocab)
    #print(counts)
    vocab = sorted(counts, key=counts.get, reverse=True)
    word_to_id = { word : i for i, word in enumerate(vocab)}
    id_to_word = {i:word for i,word in enumerate(vocab)}
    return word_to_id, id_to_word

In [21]:
vocab = ' '.join(sub_data['content']).split()
vocab.append('unknown')
word_to_id, id_to_word = word_to_id(vocab)

In [22]:
type(vocab)

list

In [23]:
def comment_to_id(word_to_id,comments):
    comment_to_id = []
    for comment in comments:
        comment_to_id.append([word_to_id[word] for word in comment.split()] )
    return comment_to_id

In [24]:
comment_to_id = comment_to_id(word_to_id,sub_data['content'])

In [25]:
def pad_sequences(comment_to_id,maxlen,padding,truncating):
    features = np.zeros((len(comment_to_id), maxlen), dtype=int)
    for i,comment in enumerate(comment_to_id):
        if len(comment) <= maxlen and padding == 'pre':
            features[i, -len(comment):] = np.array(comment)[:maxlen]
        if len(comment) <= maxlen and padding == 'post':
            features[i, :len(comment)] = np.array(comment)[:maxlen]
        if len(comment) > maxlen and truncating == 'post':
            features[i, :] = np.array(comment)[:maxlen]
        if len(comment) > maxlen and truncating == 'pre':
            features[i, :] = np.array(comment)[len(comment)-maxlen:]           
    return features

In [26]:
pad_comments = pad_sequences(comment_to_id,maxlen=200,padding='post',truncating='post')

In [27]:
pad_comments.shape

(10000, 200)

In [28]:
data.columns

Index(['id', 'content', 'location_traffic_convenience',
       'location_distance_from_business_district', 'location_easy_to_find',
       'service_wait_time', 'service_waiters_attitude',
       'service_parking_convenience', 'service_serving_speed', 'price_level',
       'price_cost_effective', 'price_discount', 'environment_decoration',
       'environment_noise', 'environment_space', 'environment_cleaness',
       'dish_portion', 'dish_taste', 'dish_look', 'dish_recommendation',
       'others_overall_experience', 'others_willing_to_consume_again'],
      dtype='object')

In [29]:
labels = sub_data[[ 'location_traffic_convenience',
       'location_distance_from_business_district', 'location_easy_to_find',
       'service_wait_time', 'service_waiters_attitude',
       'service_parking_convenience', 'service_serving_speed', 'price_level',
       'price_cost_effective', 'price_discount', 'environment_decoration',
       'environment_noise', 'environment_space', 'environment_cleaness',
       'dish_portion', 'dish_taste', 'dish_look', 'dish_recommendation',
       'others_overall_experience', 'others_willing_to_consume_again']].values

In [30]:
def dig_lists(l):
    output = []
    for e in l:
        if isinstance(e, list):
            output += dig_lists(e)
        else:
            output.append(e)
    return(output)

In [31]:
answer ={-2:[1,0,0,0],-1:[0,1,0,0],0:[0,0,1,0],1:[0,0,0,1]}


y_test = []
for line in labels:
    lineTemp = []
    for item in line:        
        lineTemp.append(answer[item])   
                       
    l2=dig_lists(lineTemp)
    y_test.append(l2)

In [32]:
for i in range(len(labels)):
    for j in range(20):
        labels[i][j]+=2

In [33]:
len(y_test[5])

80

In [34]:
def convert_2_labels(answer):
    labels_input = []
    for item in answer:
        answer_temp = []
        for i in range(0,80,4):
            answer_temp.append(np.argmax(item[i:i+4]))
        labels_input.append(answer_temp)
    return labels_input



In [35]:
def split_dataset(pad_comments,labels,split_frac):
    split_index = int(len(pad_comments)*split_frac)
    data_list = list(zip(pad_comments, labels))
    random.shuffle(data_list)
    pad_comments, labels = zip(*data_list)
    x_train, x_test = pad_comments[:split_index], pad_comments[split_index:]
    y_train, y_test = labels[:split_index], labels[split_index:]
    return x_train,y_train,x_test,y_test 

In [36]:
temp_answer = convert_2_labels(y_test)

In [37]:
temp_answer[8]

[0, 0, 0, 0, 0, 0, 0, 2, 3, 2, 3, 3, 3, 3, 3, 2, 0, 0, 3, 3]

In [38]:
 labels[8]

array([0, 0, 0, 0, 0, 0, 0, 2, 3, 2, 3, 3, 3, 3, 3, 2, 0, 0, 3, 3],
      dtype=int64)

In [39]:
for i in range(0,19):
    if temp_answer[0][i] == labels[0][i]:
        print('same')

same
same
same
same
same
same
same
same
same
same
same
same
same
same
same
same
same
same
same


In [40]:
import random
x_train,y_train,x_test,y_test = split_dataset(pad_comments,y_test,0.8)

In [41]:
len(x_train[0])

200

# Model

In [42]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.merge import add
from keras.layers import LSTM,Bidirectional, Dropout,GRU
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras import regularizers

In [43]:
max_comment_length = 200
embedding_vecor_length = 300

In [44]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test) 

In [45]:
"""Baseline model"""
def get_model():
    embed_size = 128
    inp = Input(shape=(200, ))
    x = Embedding(90000, embed_size)(inp)
    x = Bidirectional(LSTM(50, return_sequences=True))(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.2)(x)
    x = Dense(100, activation="relu")(x)
    x = Dropout(0.2)(x)
    x = Dense(4, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [52]:
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
def get_model():
    embed_size=128
    sequence_input = Input(shape=(200, ))
    x = Embedding(90000, embed_size, trainable = False)(sequence_input)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(GRU(128, return_sequences=True,dropout=0.1,recurrent_dropout=0.2))(x)
    x = Conv1D(32, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, max_pool]) 
    # x = Dense(128, activation='relu')(x)
    # x = Dropout(0.1)(x)
    preds = Dense(80, activation="sigmoid")(x)
    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',optimizer=Adam(lr=1e-3),metrics=['accuracy'])
    return model

In [53]:
model = get_model()

In [ ]:
print(model.summary())
history = model.fit(x_train, y_train,validation_data=(x_test, y_test), epochs=5,verbose=1, batch_size=40)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 200, 128)     11520000    input_3[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_3 (SpatialDro (None, 200, 128)     0           embedding_3[0][0]                
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 200, 256)     197376      spatial_dropout1d_3[0][0]        
__________________________________________________________________________________________________
conv1d_3 (

model = Sequential()
model.add(Embedding(len(word_to_id), embedding_vecor_length, input_length=max_comment_length, dropout=0.2))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(50, return_sequences=True)))
model.add(Bidirectional(LSTM(10)))
model.add(Dropout(0.5))
model.add(Dense(80, activation='sigmoid', activity_regularizer=regularizers.l2(0.001)))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
history = model.fit(x_train, y_train,validation_data=(x_test, y_test), epochs=5,verbose=1, batch_size=1000)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
print(history.history.keys())
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

preds = model.predict(x_train)
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])
print(accuracy(preds,y_train))

In [ ]:
preds

In [ ]:
convert_2_labels(preds)

In [ ]:
convert_2_labels(y_test)

In [ ]:
y_train

In [ ]:
coun=0
all1=0
for line in convert_2_labels(preds):
    for item in line:
        if item == 1:
            coun+=1
        all1+=1

In [ ]:
coun/all1

In [ ]:
hahahahaha

In [ ]:
class DataLoader():
    def __init__(self,filename,sampleNum):
        self.filename = filename
        self.data = self.load_data()
        random_indices = np.random.choice(np.arange(len(self.data['content'])), sampleNum)
        self.sub_data = self.data.iloc[random_indices]
        self.sub_data['content'] = self.sub_data['content'].apply(self.clean_sentence)

    def load_data(self):
        return pd.read_csv(self.filename,encoding='UTF-8')

    def is_CN_char(self,ch):
        return ch >= u'\u4e00' and ch <= u'\u9fa5'

    def cut(self,string):
        return list(jieba.cut(string))

    def get_stopwords(self,filename = "D:/senior/aiCourse/dataSource/stop_word.txt"):
        stopwords_dic = open(filename, encoding= 'utf-8')
        stopwords = stopwords_dic.readlines()
        stopwords = [w.strip() for w in stopwords]
        stopwords_dic.close()
        return stopwords

    def convert2simple(self,word):
        openCC = OpenCC('tw2sp')
        return openCC.convert(word)

    def clean_sentence(self,sentence):
        stopwords = self.get_stopwords()
        sentence = ''.join(filter(self.is_CN_char,sentence))
        #sentence = convert2simple(sentence)
        words = [w for w in self.cut(sentence) if len(w)>1 and w not in stopwords]
        words = ' '.join(words)
        return words


trainData = DataLoader('D:/senior/aiCourse/dataSource/comment_classification/train/sentiment_analysis_trainingset.csv',5000)


In [ ]:
testData = DataLoader('D:/senior/aiCourse/dataSource/comment_classification/test/sentiment_analysis_testa.csv',1000)

In [ ]:
class data2vec():
    def __init__(self,inputData):

        self.vocab = ' '.join(inputData['content']).split()
        
        self.vocab.append('unknown')
        
        self.word_to_id, self.id_to_word = self.word_to_id(self.vocab)

        self.comment_to_id = self.comment_to_id(self.word_to_id,inputData['content'])

        self.pad_comments = self.pad_sequences(self.comment_to_id,maxlen=200,padding='post',truncating='post')
        
    def word_to_id(self,vocab):
        counts = Counter(vocab)
        vocab = sorted(counts, key=counts.get, reverse=True)
        word_to_id = { word : i for i, word in enumerate(vocab)}
        id_to_word = {i:word for i,word in enumerate(vocab)}
        return word_to_id, id_to_word


    def comment_to_id(self,word_to_id,comments):
        comment_to_id = []
        for comment in comments:
            comment_to_id.append([word_to_id[word] for word in comment.split()] )
        return comment_to_id


    def pad_sequences(self,comment_to_id,maxlen,padding='post',truncating='post'):
        features = np.zeros((len(comment_to_id), maxlen), dtype=int)
        for i,comment in enumerate(comment_to_id):
            if len(comment) <= maxlen and padding == 'pre':
                features[i, -len(comment):] = np.array(comment)[:maxlen]
            if len(comment) <= maxlen and padding == 'post':
                features[i, :len(comment)] = np.array(comment)[:maxlen]
            if len(comment) > maxlen and truncating == 'post':
                features[i, :] = np.array(comment)[:maxlen]
            if len(comment) > maxlen and truncating == 'pre':
                features[i, :] = np.array(comment)[len(comment)-maxlen:]
        return features

In [ ]:
trainDataId = data2vec(trainData.sub_data)
testDataId = data2vec(testData.sub_data)

In [ ]:
y_train = trainData.sub_data[[ 'location_traffic_convenience',
       'location_distance_from_business_district', 'location_easy_to_find',
       'service_wait_time', 'service_waiters_attitude',
       'service_parking_convenience', 'service_serving_speed', 'price_level',
       'price_cost_effective', 'price_discount', 'environment_decoration',
       'environment_noise', 'environment_space', 'environment_cleaness',
       'dish_portion', 'dish_taste', 'dish_look', 'dish_recommendation',
       'others_overall_experience', 'others_willing_to_consume_again']].values

y_test =  testData.sub_data[[ 'location_traffic_convenience',
       'location_distance_from_business_district', 'location_easy_to_find',
       'service_wait_time', 'service_waiters_attitude',
       'service_parking_convenience', 'service_serving_speed', 'price_level',
       'price_cost_effective', 'price_discount', 'environment_decoration',
       'environment_noise', 'environment_space', 'environment_cleaness',
       'dish_portion', 'dish_taste', 'dish_look', 'dish_recommendation',
       'others_overall_experience', 'others_willing_to_consume_again']].values

for i in range(len(y_test)):
    for j in range(20):
        y_test[i][j]+=2

x_train,y_train,x_test,y_test = split_dataset(pad_comments,labels,0.8)